In [1]:
# Imports for the 
import re
import csv
import sqlite3
from pathlib import Path
import pandas as pd

In [2]:
# Create database file and establish a connection
database_path = "./MPro_database.db"
conn = sqlite3.connect(database_path)
drop_order = ('compounds', 'assays')

In [3]:
# Create a database of compounds and their SMILES string
conn.execute('''
CREATE TABLE compounds
(
    CID PRIMARY KEY,
    SMILES
)
        ''')

In [4]:
df = pd.read_csv('activity_data.csv')

for i in range(len(df)):
    row = df.iloc[i]
    values  =  (row['CID'], 
         row['SMILES']
        )
    conn.execute('INSERT INTO compounds (CID, SMILES) VALUES (?,?)',
        values)

In [5]:
# Creates and populates a table of assays, including a foreign key linking the assays to the compounds table
conn.execute('''
CREATE TABLE assays
(
    CID PRIMARY KEY,
    canonical_CID,
    r_inhibition_at_20_uM,
    r_inhibition_at_50_uM,
    r_avg_IC50,
    f_inhibition_at_20_uM,
    f_inhibition_at_50_uM,
    f_avg_IC50,
    f_avg_pIC50,
    relative_solubility_at_20_uM,
    relative_solubility_at_100_uM,
    trypsin_IC50,
    NMR_std_ratio,
    acrylamide,
    chloroacetamide,
    series,
    frag_id,
    compoundID,
    FOREIGN KEY (CompoundID) REFERENCES compounds (CID)
)
        ''')


In [7]:
df_short = df.iloc[:,1:]

for i in range(len(df_short)):
    row = df_short.iloc[i]
    values = []
    
    for element in row[:]:
        values.append(element)
    conn.execute('INSERT INTO assays (CID, canonical_CID, r_inhibition_at_20_uM, r_inhibition_at_50_uM, r_avg_IC50, f_inhibition_at_20_uM, f_inhibition_at_50_uM, f_avg_IC50, f_avg_pIC50, relative_solubility_at_20_uM, relative_solubility_at_100_uM, trypsin_IC50, NMR_std_ratio, acrylamide, chloroacetamide, series, frag_id) VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)',
             (values)
)

('MAT-POS-be048f2c-5', 'MAT-POS-be048f2c-5', None, None, None, None, None, 0.297012069374338, None, None, None, None, None, 0, 0, '3-aminopyridine-like', None, None)
('EDJ-MED-cc48ee33-4', 'EDJ-MED-cc48ee33-4', None, None, None, None, None, 0.682987814617111, None, None, None, None, None, 0, 0, '3-aminopyridine-like', None, None)
('EDJ-MED-cc48ee33-5', 'EDJ-MED-cc48ee33-5', None, None, None, None, None, 0.463982011635883, None, None, None, None, None, 0, 0, '3-aminopyridine-like', None, None)
('MIK-UNK-58cd43d0-1', 'MIK-UNK-58cd43d0-1', None, None, None, None, None, 0.723364706056088, None, None, None, None, None, 0, 0, '3-aminopyridine-like', None, None)
('MIK-UNK-78dbf1b8-1', 'MIK-UNK-78dbf1b8-1', None, None, None, None, None, 0.205138508118445, None, None, None, None, None, 0, 0, '3-aminopyridine-like', None, None)
('SAL-UNI-60119594-1', 'SAL-UNI-60119594-1', None, None, None, None, None, 99.5, None, None, None, None, None, 0, 0, None, None, None)
('SAL-UNI-60119594-2', 'SAL-UNI-601

In [11]:
def retrieve_SMILES(compoundID: str):
    list_smiles = []
    SMILES = conn.execute('''
        SELECT SMILES
        FROM compounds
        WHERE 
            CID = ?
    ''', (compoundID,))
    for sm in SMILES:
        list_smiles.append(sm)
    return (list_smiles)
        
def retrieve_assays(compoundID: str):
    list_assays = []
    ASSAYS = conn.execute('''
        SELECT series
        FROM assays
        WHERE 
            compoundID = ?
    ''', (compoundID,))
    
    for assay in ASSAYS:
        print(assay)
        list_assays.append(assay)
    return (list_assays)
    
retrieve_assays('MAT-POS-be048f2c-5')


[]

In [ ]:
# Code to display the tables currently in the database

cur = conn.execute('''
            SELECT 
                name
            FROM 
                sqlite_master 
            WHERE 
                type ='table' AND 
                name NOT LIKE 'sqlite_%'
        ''')

tables = []

for row in cur.fetchall():
    tables.append(row[0])

for table in tables:
    cur = conn.execute('SELECT  sql  FROM  sqlite_master  WHERE name=\'' + table + '\'')

    row = cur.fetchone()

    if row is not None:
        print(table+'\n')

        print(row)
    else:
        print('Table ' + table + ' not found')


In [ ]:
#for table_name in drop_order:
 #   conn.execute('DROP TABLE IF EXISTS ' + table_name)